# Práctica 1: Algoritmo RC4
*CC. Dra. Rocío Aldeco-Pérez.*

RC4 es un cifrado de flujo famoso por ser muy simple y rápido a nivel de software. Se han encontrado varias vulnerabilidades en RC4, lo que lo hace vulnerable especialmente cuando no se descarta el comienzo del flujo de clave de salida, o cuando se utilizan claves no aleatorias o relacionadas. Por esta razón, en 2015 IETF publicó el RFC 7465 para prohibir el uso de RC4 en TLS (un protocolo muy importante utilizado por los navegadores de Internet para asegurar las comunicaciones).

RC4 genera un flujo de bits pseudoaleatorios (llamado flujo de clave). Como con cualquier cifrado de flujo, estos pueden ser utilizados para el cifrado combinándolo con el texto plano utilizando la operación XOR a nivel de bits. El descifrado se realiza de la misma manera (ya que XOR con los datos dados es una involución). Para generar el flujo de clave, el cifrado utiliza un estado interno secreto que consta de dos partes:

1. Una permutación de los 256 posibles bytes (denominada "S" a continuación).
2. Dos punteros de índice de 8 bits (denominados "i" y "j").

La permutación se inicializa con una clave de longitud variable, típicamente entre 40 y 256 bits, utilizando el algoritmo de programación de clave (KSA). Una vez que esto se ha completado, se genera el flujo de bits utilizando el algoritmo de generación pseudoaleatorio (PRGA). Luego, este flujo de bits se combina mediante XOR con el texto plano dado.

In [49]:
# Key Scheduling Algorithm en Python.

def KSA(key):
    """A partir de una clave dada genera una subclave de 256 bytes."""
    length = len(key)
    s = list(range(256))
    j = 0
    for i in range(0, 256):
        j = (j + s[i] + key[i % length]) % 256
        s[i], s[j] = s[j], s[i]
    return s

# Algoritmo de generación Pseudo-aleatoria

El byte de salida se selecciona buscando los valores de S(i) y S(j), sumándolos módulo 256 y luego utilizando la suma como índice en S; S(S(i) + S(j)) se utiliza como un byte del flujo de clave, K.

Durante tantas iteraciones como se necesiten, PRGA modifica el estado y produce un byte del flujo de clave. En cada iteración, la PRGA incrementa i, busca el elemento i-ésimo de S, S[i], y le suma j, intercambia los valores de S[i] y S[j], y luego utiliza la suma S[i] + S[j] (módulo 256) como índice para obtener un tercer elemento de S (el valor del flujo de clave K), que se XORéa con el siguiente byte del mensaje para producir el siguiente byte de cifrado o texto plano. Cada elemento de S se intercambia con otro elemento al menos una vez cada 256 iteraciones.

```python
i := 0
j := 0
while GeneratingOutput:
    i := (i + 1) mod 256
    j := (j + S[i]) mod 256
    swap values of S[i] and S[j]
    K := S[(S[i] + S[j]) mod 256]
    output K
endwhile
``` 

 Ahora veamos su implementación en Python:

# El algoritmo de programación de clave (KSA, Key Scheduling Algorithm)

El algoritmo de programación de clave se utiliza para inicializar la permutación en la matriz "S". "keylength" se define como el número de bytes en la clave y puede estar en el rango 1 ≤ keylength ≤ 256, típicamente entre 5 y 16, correspondiente a una longitud de clave de 40 a 128 bits. En primer lugar, la matriz "S" se inicializa en la permutación de identidad. Luego, se procesa S durante 256 iteraciones de manera similar a la PRGA principal, pero también mezcla bytes de la clave al mismo tiempo. E

Esto en pseudocódigo resultaría como:
 ``` 
for i from 0 to 255
S[i] := i
endfor
j := 0
for i from 0 to 255
    j := (j + S[i] + key[i mod keylength]) mod 256
    swap values of S[i] and S[j]
endfor
 ``` 

 Ahora veamos su equivalente en Python:

In [50]:
def PRGA(s):
    """A partir de la permutación S de 256 bytes genera un stream pseudoaleatorio."""
    i = 0
    j = 0
    while True:
        i = (i + 1) % 256
        j = (j + s[i]) % 256
        s[i], s[j] = s[j], s[i] # Intercambia valores entre S[i] y S[j]
        K = s[(s[i] + s[j]) % 256]
        yield K # Generador a la salida conforme se necesiten bits.

# Probando funcionamiento con vectores de prueba

Ahora, utilizaremos los siguientes vectores de prueba para verificar el funcionamiento de la implementación de RC4 realizada:

| Clave | Flujo de clave | Texto plano | Criptograma
| --- | --- | --- | --- |
| Key | EB9F7781B734CA72A719 | Plaintext | BBF316E8D940AF0AD3 |
| Wiki | 6044DB6D41B7 | pedia | 1021BF0420 |
| Secret | 04D46B053CA87B59 | Attack at dawn | 45A01F645FC35B383552544B9BF5 |

Generaremos una función adicional que implemente los bloques anteriormente descritos y la probaremos con los casos de prueba:

In [60]:
def RC4 (clave, textoplano):
    """Realiza el cifrado con RC4 de un mensaje en claro."""
    s = KSA(clave)
    flujoclave = PRGA(s)
    cifrado = bytes([p ^ next(flujoclave) for p in textoplano])
    return cifrado.hex().upper()

# Transformamos todos a un flujo de bits hexadecimal. 
key1 = "Key".encode('utf-8')
key1 = bytes.fromhex(key1.hex().upper())
text1 = "Plaintext".encode('utf-8')
text1 = bytes.fromhex(text1.hex().upper())

key2 = "Wiki".encode('utf-8')
key2 = bytes.fromhex(key2.hex().upper())
text2 = "pedia".encode('utf-8')
text2 = bytes.fromhex(text2.hex().upper())

key3 = "Secret".encode('utf-8')
key3 = bytes.fromhex(key3.hex().upper())
text3 = "Attack at dawn".encode('utf-8')
text3 = bytes.fromhex(text3.hex().upper())

print("Claves en Hexadecimal:", key1, key2, key3)
print("Textos claros en Hexadecimal:", text1, text2, text3)

# Pruebas unitarias.
print("\nIniciando cifrado con RC4...\n")

assert RC4(key1, text1) == 'BBF316E8D940AF0AD3'
print("Pass! Resultado:", RC4(key1, text1))
assert RC4(key2, text2) == '1021BF0420'
print("Pass! Resultado:", RC4(key2, text2))
assert RC4(key3, text3) == '45A01F645FC35B383552544B9BF5'
print("Pass! Resultado:", RC4(key3, text3))

Claves en Hexadecimal: b'Key' b'Wiki' b'Secret'
Textos claros en Hexadecimal: b'Plaintext' b'pedia' b'Attack at dawn'

Iniciando cifrado con RC4...

Pass! Resultado: BBF316E8D940AF0AD3
Pass! Resultado: 1021BF0420
Pass! Resultado: 45A01F645FC35B383552544B9BF5
